# Sistema Criptográfico para Criação de Contratos em Redes Públicas

In [1]:
# Imports
from cryptolib_api import *
from uuid import uuid4

In [2]:
# Clear DB to remove inconsistencies and crate users
clear_data()

user1 = create_user('user1', 'pass1')
user2 = create_user('user2', 'pass2')
user3 = create_user('user3', 'pass3')

pk1 = user1.get_public_key()
pk2 = user2.get_public_key()
pk3 = user3.get_public_key()

In [3]:
# Define asset id and amount to be created
asset_id = uuid4().int
asset_amount = 1000

In [4]:
# Attempt to make operations while not logged in
create_asset(asset_id, asset_amount)

User not logged in.


In [5]:
# Log in and check user assets
login('user1', 'pass1')
get_unspent_assets()

[]

In [6]:
# Repeat operations while logged in
create_asset(asset_id, asset_amount)

Asset created.


In [7]:
# List all assets user have
list_assets()

Asset 299705440608112187293368900407592760131: 1000


In [8]:
# Check user assets
assets = get_unspent_assets()

for asset in assets:
    print("Asset(ID : {}, Amount : {})".format(asset.T, asset.amount))

Asset(ID : 299705440608112187293368900407592760131, Amount : 1000)


In [9]:
# Make first transaction sending 50 units of the asset to user2 and 100 units to user3
transaction = make_transaction(asset_id, {pk2 : 200, pk3 : 300})

Transfer successful.


In [10]:
# Check if transaction subtracted user assets
list_assets()

Asset 299705440608112187293368900407592760131: 500


In [11]:
# List TXO in transaction and check which ones bellongs to user1
images = image_pool.get_list()
text = " This TXO belongs to currently logged in user."

for txo in transaction.outputs:
    print("Sent {} of asset {}.{}".format(txo.amount, txo.T, text if check_txo_ownership(txo, images) else ""))

Sent 300 of asset 299705440608112187293368900407592760131.
Sent 200 of asset 299705440608112187293368900407592760131.
Sent 500 of asset 299705440608112187293368900407592760131. This TXO belongs to currently logged in user.


In [12]:
# Try to transfer more than user have
make_transaction(asset_id, {pk2 : 500, pk3 : 500})

Not enough to transfer.


In [13]:
# Checks user1 funds after transaction
list_assets()

Asset 299705440608112187293368900407592760131: 500


In [14]:
# Transfer all funds to not existent account
random_key = PrivateKey.create()
lost_transaction = make_transaction(asset_id, {random_key.get_public_key() : 500})

Transfer successful.


In [15]:
# Checks that after user1 transfers all its funds it has no assets
list_assets()

No assets!


In [16]:
# After transaction the user do not have ownership of the TXO anymore
images = image_pool.get_list()
text = " This TXO belongs to current logged in user."

for txo in transaction.outputs:
    print("Sent {} of asset {}.{}".format(txo.amount, txo.T, text if check_txo_ownership(txo, images) else ""))

Sent 300 of asset 299705440608112187293368900407592760131.
Sent 200 of asset 299705440608112187293368900407592760131.
Sent 500 of asset 299705440608112187293368900407592760131.


# Confirming Transactions

In [17]:
# Storing TXOs in convinient way to check receipts
def get_owner_pk(txo):
    for user in [user1, user2, user3]:
        if check_txo_ownership(txo, [], owner = user):
            return user.get_public_key()
        
txos = {get_owner_pk(txo) : txo for txo in transaction.outputs}
receipts = {pk : generate_asset_receipt(txo) for pk, txo in txos.items()}

In [18]:
# Verify that TXO that user1 received was in the transaction signed by user1
login('user1', 'pass1')
verify_asset_receipt(receipts[pk1], pk1, txos[pk1])

True

In [19]:
# Verify that TXO that user2 received was in the transaction signed by user1
login('user2', 'pass2')
list_assets()
verify_asset_receipt(receipts[pk2], pk1, txos[pk2])

Asset 299705440608112187293368900407592760131: 200


True

In [20]:
# Verify that TXO that user3 received was in the transaction signed by user1
login('user3', 'pass3')
list_assets()
verify_asset_receipt(receipts[pk3], pk1, txos[pk3])

Asset 299705440608112187293368900407592760131: 300


True

In [21]:
# Create a ownership receipt for the TXO received by user3
receipt = generate_ownership_receipt(txos[pk3])

In [22]:
# user1 verifies that receipt is indeed valid
login('user1', 'pass1')
verify_ownership_receipt(receipt)

True

In [23]:
# user3 spends half of the TXO that create the receipt
login('user3', 'pass3')
new_transaction = make_transaction(asset_id, {pk2 : 50})

Transfer successful.


In [24]:
# Checks that ownership receipt is no longer valid
login('user1', 'pass1')
verify_ownership_receipt(receipt)

False

# Ring Signature in Action

Ring signature has advantages when we have a pool of different TXO with same value

In [25]:
# Create several transactions with same value

login('user2', 'pass2')
for i in range(3):
    make_transaction(asset_id, {pk1 : 10, pk3: 10})
    
login('user3', 'pass3')
for i in range(3):
    make_transaction(asset_id, {pk1 : 10, pk2: 10})

Transfer successful.
Transfer successful.
Transfer successful.
Transfer successful.
Transfer successful.
Transfer successful.


In [26]:
login('user1', 'pass1')
transaction = make_transaction(asset_id, {pk2 : 10})

for txo in transaction.inputs:
    print("Ring size is {}.".format(len(txo.ring.txos)))

Transfer successful.
Ring size is 5.


# Data Structures

In [27]:
# PublicKey structure
pk1.serialize()

{'A': {'x': '1342268358528486480123067609685606339371703659379008892268898404963402032502',
  'y': '73443307102308914644672844249254822379579519531462498185418317786989578070508'},
 'B': {'x': '15853103917438824881513652190670553702992406799436137422113408992363202097565',
  'y': '75665670941252936516236557423900893380499529325146228045709730253495679698937'}}

In [28]:
# PublicKey structure
user1.private_key.serialize()

{'a': '86664425604820153314150265402099491126621475242464788642012889737580411429090',
 'b': '7276877179678413446309538081443363474595308394147118882526103292072109629753'}

In [29]:
# TXO structure
transaction.outputs[0].serialize()

{'P': {'x': '19541409669534735111201639254227287684406346752928778819187590674471550651952',
  'y': '46312656761378810165121440560704169081900897478027726083477665606110663818835'},
 'R': {'x': '30489812139653540367655943376042896908876051896778895103150581668805172887513',
  'y': '6192996397019570047942684076172565338446454345735985534454885494867898880137'},
 'T': '299705440608112187293368900407592760131',
 'amount': '10',
 'public_key': {'A': {'x': '31097145751655351317054021362961052938193767475791744576610222579545968110096',
   'y': '60091427898207116713995537428244620752747595534541755141186472081275075408732'},
  'B': {'x': '14766719569609798477703470378897119363157079038401356828404989377656403265860',
   'y': '49326487753204907304161408958621232325272030718329946380706156185769516158039'}}}

In [30]:
# Ring structure
transaction.inputs[0].ring.serialize()

{'txos': [{'P': {'x': '17703130624402698906860948767840831263344974302964011915231950730477642396755',
    'y': '112958177462456347570334810275262157196434184057039060320970618044011205584179'},
   'R': {'x': '32614185446506587743928041717834962208093437329519448391661122708736444841934',
    'y': '67250866790353973955241248803097359512128182152319241203573525409488811433767'},
   'T': '299705440608112187293368900407592760131',
   'amount': '10',
   'public_key': {'A': {'x': '1342268358528486480123067609685606339371703659379008892268898404963402032502',
     'y': '73443307102308914644672844249254822379579519531462498185418317786989578070508'},
    'B': {'x': '15853103917438824881513652190670553702992406799436137422113408992363202097565',
     'y': '75665670941252936516236557423900893380499529325146228045709730253495679698937'}}},
  {'P': {'x': '48051124251305589778833463972610767635384365511933399897649569571512643395211',
    'y': '23851032331772603556069885200290931619939039675523521

In [31]:
# Signature structure
transaction.inputs[0].serialize()

{'cs': ['19627859895746797832001928578851309536170169200707808609102410233600696875676',
  '10119772107982637558175269625359370803723819770790422787504935558333703992914',
  '62649380692353198630805782768533968386366939140906404103687079625104841009391',
  '105359601068489998772867595310900411607527503695505849066484034606903896241301',
  '58895896446158539521288267151769682704723926134691030529556978417009925922092'],
 'image': {'x': '67667457447356915992283390805661552626031170926461189106355724900561401137456',
  'y': '102391200456689880976278023280425924803468124707812257806289309588532185662234'},
 'ring': {'txos': [{'P': {'x': '17703130624402698906860948767840831263344974302964011915231950730477642396755',
     'y': '112958177462456347570334810275262157196434184057039060320970618044011205584179'},
    'R': {'x': '32614185446506587743928041717834962208093437329519448391661122708736444841934',
     'y': '67250866790353973955241248803097359512128182152319241203573525409488811433767'

In [32]:
# Transaction structure
transaction.serialize()

{'R': {'x': '30489812139653540367655943376042896908876051896778895103150581668805172887513',
  'y': '6192996397019570047942684076172565338446454345735985534454885494867898880137'},
 'T': '299705440608112187293368900407592760131',
 'inputs': [{'cs': ['19627859895746797832001928578851309536170169200707808609102410233600696875676',
    '10119772107982637558175269625359370803723819770790422787504935558333703992914',
    '62649380692353198630805782768533968386366939140906404103687079625104841009391',
    '105359601068489998772867595310900411607527503695505849066484034606903896241301',
    '58895896446158539521288267151769682704723926134691030529556978417009925922092'],
   'image': {'x': '67667457447356915992283390805661552626031170926461189106355724900561401137456',
    'y': '102391200456689880976278023280425924803468124707812257806289309588532185662234'},
   'ring': {'txos': [{'P': {'x': '17703130624402698906860948767840831263344974302964011915231950730477642396755',
       'y': '112958177